<img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/Digital Technical Engagement.png">

# IBM Cloud Pak for Data - Data Virtualization REST Services Class

### Where to find this notebook online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/CPDDVLAB.

### What is notebook does
This notebook is reusable class library to interact with the core RESTful services for Data Virtualiztion

In [4]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, api, userid, password):
        
        credentials = {'username':userid, 'password':password}
        r = requests.post(self.url+api+'/preauth/signin', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = "Bearer " + r.cookies["ibm-private-cloud-session"]
            print('Token Retrieved')
            self.headers = {'Content-Type':"application/json", 'Accept':"application/json", 'Authorization': bearerToken, 'Cache-Control': "no-cache"}
        else:
            print ('Unable to authenticate, no bearer token obtained')
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def deleteRequest(self, api, json=None):
        return requests.delete(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def getVirtualizedTables(self):
        return self.getRequest('/icp4data-databases/dv/cpd-instance/dvapiserver/v1/mydata/tables')
    
    def getVirtualizedTablesDF(self):
        r = self.getVirtualizedTables()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            df = pd.DataFrame(json_normalize(json['tables']))
            return df
        else:
            print(self.getStatusCode(r))
            
    def getVirtualizedViews(self):
        return self.getRequest('/icp4data-databases/dv/cpd-instance/dvapiserver/v1/mydata/views')
    
    def getVirtualizedViewsDF(self):
        r = self.getVirtualizedViews()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            df = pd.DataFrame(json_normalize(json['views']))
            return df
        else:
            print(self.getStatusCode(r))
    
    def grantPrivledgeToRole(self, objectName, objectSchema, roleToGrant):
        json =   {"objectName":objectName,"objectSchema":objectSchema,"roleToGrant":roleToGrant}
        return self.postRequest('/icp4data-databases/dv/cpd-instance/dvapiserver/v1/privileges/roles',json);
 
    def getRole(self, role):
        return self.getRequest('/icp4data-databases/dv/cpd-instance/dvapiserver/v1/privileges/objects/role/'+str(role));
    
    def foldData(self, sourceName, sourceTableDef, sources ):
        json = {"sourceName":sourceName,"sourceTableDef":sourceTableDef,"sources":sources}
        return self.postRequest('/icp4data-databases/dv/cpd-instance/dvapiserver/v1/virtualize/tables', json);

    def addUser(self, username, displayName, email, user_roles, password):
        json = {"username":username,"displayName":displayName,"email":email,"user_roles":user_roles,"password":password}
        return self.postRequest('/api/v1/usermgmt/v1/user', json);
    
    def dropUser(self, username):
        return self.deleteRequest('/api/v1/usermgmt/v1/user/'+str(username));
   
    def getUsers(self):
        return self.getRequest('/api/v1/usermgmt/v1/usermgmt/users');
    
    def getUsersDF(self):
        r = self.getUsers()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            df = pd.DataFrame(json_normalize(json))
            return df
        else:
            print(self.getStatusCode(r));
    
    def addUserToDV(self, display_name, role, usersDF):
        userrow = (usersDF.loc[usersDF['displayName'] == display_name])
        uid = userrow['uid'].values[0]
        username = userrow['username'].values[0]
        
        json = {"users":[{"uid":uid,"username":username,"display_name":display_name,"role":role}],"serviceInstanceID":"1635944153872816"}
        return self.postRequest('/zen-data/v2/serviceInstance/users', json);
    
    def dropUserFromDV(self, display_name, usersDF):
        userrow = (usersDF.loc[usersDF['displayName'] == display_name])
        uid = userrow['uid'].values[0]
        
        json = {"users":[uid],"serviceInstanceID":"1635944153872816"}
        return self.deleteRequest('/zen-data/v2/serviceInstance/users', json);
    
    def deleteVirtualizedTable(self, table_schema, table_name, data_source_table_name):
        payload = {"table_schema":table_schema,"table_name":table_name,"data_source_table_name":data_source_table_name}
        return self.deleteRequest('/icpd-instanceata-databases/dv/cpd-instance/dbapi/v4/federation', payload);
    
    def deleteView(self, schema, view):
        return self.deleteRequest('/icp4data-databases/dv/cpd-instance/dbapi/v4/federation/views/'+str(schema)+'/'+str(view))

    def getDataSourcesAPI(self):
        return self.getRequest('/icp4data-databases/dv/cpd-instance/dvapiserver/v1/datasource_nodes')
    
    def getDataSources(self):
        columns = ['cid','connection_id', 'dbname', 'srchostname', 'srcport','srctype','status','usr','uri']
        dfTotal = pd.DataFrame(columns=columns)
        r = self.getDataSourcesAPI()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            df = pd.DataFrame(json_normalize(json))
            for index, row in df.iterrows():
                if row['dscount']>'0':
                    dfTotal = pd.concat([dfTotal, pd.DataFrame(json_normalize(row['dataSources']))],ignore_index=True)
            return(dfTotal[['srctype','srchostname', 'srcport', 'dbname', 'usr', 'status']])
        else:
            print(self.getStatusCode(r))   

    def getCacheDetails(self, cache):
        r = self.getRequest('/icp4data-databases/dv/cpd-instance/dv-caching/api/v1/caches/'+str(cache))
        if (self.getStatusCode(r)==200):
            return databaseAPI.getJSON(r)
        else:
            print(self.getStatusCode(r))
            print(json['message'])        
            
    def getCaches(self, type='Available'):
        # type = 'Enabled', 'Disabled', 'Deleted', 'All'
        r = self.getRequest('/icp4data-databases/dv/cpd-instance/dv-caching/api/v1/caches')
        json = databaseAPI.getJSON(r)
        if (self.getStatusCode(r)==200):
            df = pd.DataFrame(json_normalize(json['caches']))
            if (type == 'Available'):
                return df[df["state"].isin(['Enabled','Disabled','Refreshing'])]
            elif (type == 'Enabled'):
                return df[df["state"] == 'Enabled']
            elif (type == 'Disabled'):
                return df[df["state"] == 'Disabled']
            elif (type == 'Deleted'):
                return df[df["state"] == 'Deleted']
            elif (type == 'Refreshing'):
                return df[df["state"] == 'Refreshing']
        else:
            print(self.getStatusCode(r))
            print(json['message'])
    
    def enableCache(self, cache):
        r = self.postRequest('/icp4data-databases/dv/cpd-instance/dv-caching/api/v1/enable/'+str(cache));
        json = databaseAPI.getJSON(r)
        if (self.getStatusCode(r)==202):
            print('Cache: ' + cache + " enabled.")
        else:
            print(self.getStatusCode(r))
            print(json['message'])    
        
    def disableCache(self, cache):
        r = self.postRequest('/icp4data-databases/dv/cpd-instance/dv-caching/api/v1/disable/'+str(cache));
        json = databaseAPI.getJSON(r)
        if (self.getStatusCode(r)==202):
            print('Cache: ' + cache + " disabled.")
        else:
            print(self.getStatusCode(r))
            print(json['message']) 
     
    def refreshCache(self, cache):
        r = self.postRequest('/icp4data-databases/dv/cpd-instance/dv-caching/api/v1/refresh/'+str(cache)); 
        json = databaseAPI.getJSON(r)
        if (self.getStatusCode(r)==202):
            print('Cache: ' + cache + " being refreshed. Check cache status.")
        else:
            print(self.getStatusCode(r))
            print(json['message'])     

In [ ]:
from decimal import Decimal
class Timer():
    def __init__(self):
        self.totalTime = 0
        self.time = 0
        self.list = []
        
    def wallTime(self, timing):
        start = timing.find('Wall time: ') + 11
        end = timing.find(' ms', start)
        if end == -1:
            endsec = timing.find(' s', start)
            if endsec == -1:
                endmin = timing.find('min', start)
                minutes = Decimal(timing[start:endmin])
                endsec = timing.find('s', start)
                startsec = endmin+4
                seconds = Decimal(timing[startsec:endsec])
                return (minutes*60+seconds)*1000
            else:
                return Decimal(timing[start:endsec])*1000
        else:
            return Decimal(timing[start:end])
                
    def timeTotal(self):
        standardOutput = result.stdout
        print(standardOutput)
        self.time = self.wallTime(standardOutput)
        self.list.append(self.time/1000)
        self.totalTime = self.totalTime + self.time
        print("Time: " + str(self.time/1000) + " s")
        print("Total Time: " + str(self.totalTime/1000) + " s")
        
    def getTotalTime(self):
        return self.totalTime
    
    def getLastTime(self):
        return self.time  
    
    def getList(self):
        return self.list

#### Credits: IBM 2021, Peter Kohlmann [kohlmann@ca.ibm.com]